In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [3]:
import pandas as pd

df = pd.read_json(
    "https://huggingface.co/datasets/gorilla-llm/Berkeley-Function-Calling-Leaderboard/raw/main/gorilla_openfunctions_v1_test_simple.json",
    lines=True,
)
df.head()

,question,function
0,Find the area of a triangle with a base of 10 ...,"{'name': 'calculate_triangle_area', 'descripti..."
1,Calculate the factorial of 5 using math functi...,"{'name': 'math.factorial', 'description': 'Cal..."
2,Calculate the hypotenuse of a right triangle g...,"{'name': 'math.hypot', 'description': 'Calcula..."
3,Find the roots of a quadratic equation with co...,"{'name': 'algebra.quadratic_roots', 'descripti..."
4,"Solve a quadratic equation where a=2, b=6, and...","{'name': 'solve_quadratic_equation', 'descript..."


In [4]:
import llama_cpp
from llama_cpp import Llama

llm = Llama(
    "/home/asilva/models/Hermes-2-Pro-Llama-3-8B-F16.gguf",
    tokenizer=llama_cpp.llama_tokenizer.LlamaHFTokenizer.from_pretrained(
        "NousResearch/Hermes-2-Pro-Llama-3-8B"
    ),
    n_gpu_layers=-1,
    flash_attn=True,
    n_ctx=8192,
    verbose=False,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
pos = 0
question = df.iloc[pos]["question"]
question

'Find the area of a triangle with a base of 10 units and height of 5 units.'

In [6]:
import json

function = df.iloc[pos]["function"]
json.dumps(function)

'{"name": "calculate_triangle_area", "description": "Calculate the area of a triangle given its base and height.", "parameters": {"type": "dict", "properties": {"base": {"type": "integer", "description": "The base of the triangle."}, "height": {"type": "integer", "description": "The height of the triangle."}, "unit": {"type": "string", "description": "The unit of measure (defaults to \'units\' if not specified)"}}, "required": ["base", "height"]}}'

In [7]:
final_prompt = (
    "<|im_start|>system\n"
    "You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. "
    "You may call one or more functions to assist with the user query. "
    "Don't make assumptions about what values to plug into functions. "
    "Here are the available tools: <tools> ["
    + json.dumps(function)
    + "] </tools> Use the following pydantic model json schema for each tool call you will make: "
    "{'title': 'FunctionCall', 'type': 'object', 'properties': {'arguments': {'title': 'Arguments', 'type': 'object'}, 'name': {'title': 'Name', 'type': 'string'}}, 'required': ['arguments', 'name']} "
    "For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows: "
    "<tool_call>\n"
    "{'arguments': <args-dict>, 'name': <function-name>}"
    "</tool_call><|im_end|>\n"
    "<|im_start|>user\n" + question + "<|im_start|>assistant\n"
    "<tool_call>"
)
final_prompt

'<|im_start|>system\nYou are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. You may call one or more functions to assist with the user query. Don\'t make assumptions about what values to plug into functions. Here are the available tools: <tools> [{"name": "calculate_triangle_area", "description": "Calculate the area of a triangle given its base and height.", "parameters": {"type": "dict", "properties": {"base": {"type": "integer", "description": "The base of the triangle."}, "height": {"type": "integer", "description": "The height of the triangle."}, "unit": {"type": "string", "description": "The unit of measure (defaults to \'units\' if not specified)"}}, "required": ["base", "height"]}}] </tools> Use the following pydantic model json schema for each tool call you will make: {\'title\': \'FunctionCall\', \'type\': \'object\', \'properties\': {\'arguments\': {\'title\': \'Arguments\', \'type\': \'object\'}, \'name\': {\'title\': 

In [8]:
response = llm(final_prompt, temperature=0, seed=0, max_tokens=None)

In [9]:
response["choices"][0]["text"]

" {'arguments': {'base': 10, 'height': 5}, 'name': 'calculate_triangle_area'}</tool_call>"

In [10]:
import regex

def extract_first_of_nested_brackets(input_string):
    pattern = r"\{(?>[^{}]+|(?R))*\}"
    # pattern = r'\[(.*?)\]'
    match = regex.search(pattern, input_string)
    if match:
        return match.group(0)
    return None

In [11]:
final_result = extract_first_of_nested_brackets(response["choices"][0]["text"])
final_result

"{'arguments': {'base': 10, 'height': 5}, 'name': 'calculate_triangle_area'}"

In [12]:
import json_repair

json_repair.loads(final_result)

{'arguments': {'base': 10, 'height': 5}, 'name': 'calculate_triangle_area'}

In [13]:
GORILLA_TO_OPENAPI = {
    "integer": "integer",
    "number": "number",
    "float": "number",
    "string": "string",
    "boolean": "boolean",
    "bool": "boolean",
    "array": "array",
    "list": "array",
    "dict": "object",
    "object": "object",
    "tuple": "array",
    "any": "string",
    "byte": "integer",
    "short": "integer",
    "long": "integer",
    "double": "number",
    "char": "string",
    "ArrayList": "array",
    "Array": "array",
    "HashMap": "object",
    "Hashtable": "object",
    "Queue": "array",
    "Stack": "array",
    "Any": "string",
    "String": "string",
    "Bigint": "integer",
}

In [14]:
def _cast_to_openai_type(properties, mapping, test_category):
    for key, value in properties.items():
        if "type" not in value:
            properties[key]["type"] = "string"
        else:
            var_type = value["type"]
            if mapping == GORILLA_TO_OPENAPI and var_type == "float":
                properties[key]["format"] = "float"
                properties[key]["description"] += " This is a float type value."
            if var_type in mapping:
                properties[key]["type"] = mapping[var_type]
            else:
                properties[key]["type"] = "string"

        # Currently support:
        # - list of any
        # - list of list of any
        # - list of dict
        # - list of list of dict
        # - dict of any

        if properties[key]["type"] == "array" or properties[key]["type"] == "object":
            if "properties" in properties[key]:
                properties[key]["properties"] = _cast_to_openai_type(
                    properties[key]["properties"], mapping, test_category
                )
            elif "items" in properties[key]:
                properties[key]["items"]["type"] = mapping[
                    properties[key]["items"]["type"]
                ]
                if (
                    properties[key]["items"]["type"] == "array"
                    and "items" in properties[key]["items"]
                ):
                    properties[key]["items"]["items"]["type"] = mapping[
                        properties[key]["items"]["items"]["type"]
                    ]
                elif (
                    properties[key]["items"]["type"] == "object"
                    and "properties" in properties[key]["items"]
                ):
                    properties[key]["items"]["properties"] = _cast_to_openai_type(
                        properties[key]["items"]["properties"], mapping, test_category
                    )
    return properties

In [15]:
# allow `\"`, `\\`, or any character which isn't a control sequence
STRING_INNER = r'([^"\\\x00-\x1F\x7F-\x9F]|\\["\\])'
STRING = f'"{STRING_INNER}*"'

INTEGER = r"(-)?(0|[1-9][0-9]*)"
# NUMBER = rf"({INTEGER})(\.[0-9]+)?([eE][+-][0-9]+)?"
NUMBER = rf"({INTEGER})(\.[0-9]+)([eE][+-][0-9]+)?"
BOOLEAN = r"(true|false)"
NULL = r"null"
WHITESPACE = r"[ ]?"

_type_to_regex = {
    "string": STRING,
    "integer": INTEGER,
    "number": NUMBER,
    "boolean": BOOLEAN,
    "null": NULL,
}

In [16]:
function_data = function

In [17]:
for arg, value in _cast_to_openai_type(
    function_data["parameters"]["properties"], GORILLA_TO_OPENAPI, "simple"
).items():
    # do any of the examples use the option parameter?
    # Easy enough to add in!
    if arg in function_data["parameters"]["required"]:
        print(arg)
        print(value)

base
{'type': 'integer', 'description': 'The base of the triangle.'}
height
{'type': 'integer', 'description': 'The height of the triangle.'}


In [18]:
def build_dict_regex(props):
    out_re = r'\{'
    args_part  = ", ".join([
        f'"{prop}": '+type_to_regex(props[prop])
        for prop in props
    ])
    return out_re+args_part+r"\}"

In [19]:
# this should be more universal
def type_to_regex(arg_meta):
    # import pdb; pdb.set_trace()
    basic_map = {
        "string": f'"{STRING_INNER}{{1,42}}"',  # might need to be longer?
        "integer": INTEGER,
        "number": NUMBER,
        "float": NUMBER,  # change this later
        "boolean": BOOLEAN,
        "null": NULL,
    }
    arg_type = arg_meta["type"]
    if arg_type == "object":
        arg_type = "dict"
    if arg_type == "dict":
        # import pdb; pdb.set_trace()
        if "properties" in arg_meta.keys():
            result = build_dict_regex(arg_meta["properties"])
        else:
            result = r"""\[("([^"\\\x00-\x1F\x7F-\x9F]|\\["\\]){1,42}", ){0,8}"([^"\\\x00-\x1F\x7F-\x9F]|\\["\\]){1,42}"\]"""
    # Note, this currently won't pass the empty list
    elif arg_type == "array":
        pattern = type_to_regex(arg_meta["items"])
        result = array_regex = r"\[(" + pattern + ", ){0,8}" + pattern + "\]"
    else:
        result = basic_map[arg_type]
    # print(result)
    return result

In [20]:
def build_fc_regex(function_data):
    out_re = r'\{"name": "' + function_data["name"] + '", "arguments": \{'
    args_part = ", ".join(
        [
            f'"{arg}": ' + type_to_regex(value)
            for arg, value in _cast_to_openai_type(
                function_data["parameters"]["properties"], GORILLA_TO_OPENAPI, "simple"
            ).items()
            # do any of the examples use the option parameter?
            # Easy enough to add in!
            if arg in function_data["parameters"]["required"]
        ]
    )
    optional_part = "".join(
        [
            f'(, "{arg}": ' + type_to_regex(value) + r")?"
            for arg, value in _cast_to_openai_type(
                function_data["parameters"]["properties"], GORILLA_TO_OPENAPI, "simple"
            ).items()
            if not (arg in function_data["parameters"]["required"])
        ]
    )
    # return out_re + args_part + optional_part + "}}"
    return out_re + args_part + "}}"

In [21]:
function

{'name': 'calculate_triangle_area',
 'description': 'Calculate the area of a triangle given its base and height.',
 'parameters': {'type': 'dict',
  'properties': {'base': {'type': 'integer',
    'description': 'The base of the triangle.'},
   'height': {'type': 'integer', 'description': 'The height of the triangle.'},
   'unit': {'type': 'string',
    'description': "The unit of measure (defaults to 'units' if not specified)"}},
  'required': ['base', 'height']}}

In [22]:
regex = build_fc_regex(function)
regex

'\\{"name": "calculate_triangle_area", "arguments": \\{"base": (-)?(0|[1-9][0-9]*), "height": (-)?(0|[1-9][0-9]*)}}'

In [23]:
from outlines import models, generate

model = models.LlamaCpp(llm)
generator = generate.regex(model, regex)
answer = generator(final_prompt, 
                   max_tokens=1024, temperature=0, seed=42)

In [24]:
answer

'{"name": "calculate_triangle_area", "arguments": {"base": 10, "height": 5}}'

In [25]:
answers = []
for pos in range(10):
    try:
        print(pos)
        question = df.iloc[pos]["question"]
        function = df.iloc[pos]["function"]
        final_prompt = (
            "<|im_start|>system\n"
            "You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. "
            "You may call one or more functions to assist with the user query. "
            "Don't make assumptions about what values to plug into functions. "
            "Here are the available tools: <tools> ["
            + json.dumps(function)
            + "] </tools> Use the following pydantic model json schema for each tool call you will make: "
            "{'title': 'FunctionCall', 'type': 'object', 'properties': {'arguments': {'title': 'Arguments', 'type': 'object'}, 'name': {'title': 'Name', 'type': 'string'}}, 'required': ['arguments', 'name']} "
            "For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows: "
            "<tool_call>\n"
            "{'arguments': <args-dict>, 'name': <function-name>}"
            "</tool_call><|im_end|>\n"
            "<|im_start|>user\n" + question + "<|im_start|>assistant\n"
            "<tool_call>"
        )
        regex = build_fc_regex(function)
        generator = generate.regex(model, regex)
        answer = generator(final_prompt, max_tokens=1024, temperature=0, seed=0)
        answers.append(answer[1:-1])
    except:
        answers.append("Error")

0
1


Compiling FSM index for all state transitions: 100%|██████████| 60/60 [00:01<00:00, 38.58it/s]


2


Compiling FSM index for all state transitions: 100%|██████████| 61/61 [00:01<00:00, 37.95it/s]


3


...

Compiling FSM index for all state transitions: 100%|██████████| 66/66 [00:01<00:00, 39.86it/s]


8


Compiling FSM index for all state transitions: 100%|██████████| 66/66 [00:01<00:00, 38.12it/s]


9


Compiling FSM index for all state transitions: 100%|██████████| 76/76 [00:01<00:00, 40.48it/s]


In [26]:
answers

['"name": "calculate_triangle_area", "arguments": {"base": 10, "height": 5}',
 '"name": "math.factorial", "arguments": {"number": 5}',
 '"name": "math.hypot", "arguments": {"x": 4, "y": 5}',
 '"name": "algebra.quadratic_roots", "arguments": {"a": 1, "b": -3, "c": 2}',
 '"name": "solve_quadratic_equation", "arguments": {"a": 2, "b": 6, "c": 5}',
 '"name": "solve_quadratic", "arguments": {"a": 3, "b": -11, "c": -4}',
 '"name": "solve_quadratic", "arguments": {"a": 2, "b": 5, "c": 3}',
 '"name": "calculate_circumference", "arguments": {"radius": 4}',
 '"name": "geometry.area_circle", "arguments": {"radius": 10}',
 '"name": "geometry.calculate_area_circle", "arguments": {"radius": 5}']

In [27]:
len(answers)

10

In [35]:
# RUN IT ONCE TO DOWNLOAD THE TEST DATASET
# !wget https://raw.githubusercontent.com/ShishirPatil/gorilla/main/berkeley-function-call-leaderboard/data/possible_answer/gorilla_openfunctions_v1_test_simple.json

In [36]:
file_path = "gorilla_openfunctions_v1_test_simple.json"

In [37]:
# standardize possible_answers from gorilla simple test set
import json

possible_answers = []
# Open the file and read line by line
with open(file_path, "r") as file:
    for line in file:
        # Parse the JSON line into a Python dictionary
        json_data = json.loads(line)

        # Process the JSON object
        for function_name, params in json_data.items():
            final_string = function_name + ",{"
            for param_name, param_value in params.items():
                final_string += '"' + param_name + '":' + str(param_value) + ","
            final_string = final_string[:-1] + "}"
            final_string = final_string.replace("'", '"')
            final_string = final_string.replace(" ", "")
            possible_answers.append(final_string)

In [38]:
json_repair.loads("{" + answers[0] + "}")

{'name': 'calculate_triangle_area', 'arguments': {'base': 10, 'height': 5}}

In [35]:
# standardize model_output
def standardize(result):
    final_result = (
        str(json_repair.loads("{" + result + "}")["name"])
        + ","
        + str(json_repair.loads("{" + result + "}")["arguments"])
    )
    final_result = final_result.replace("'", '"')
    final_result = final_result.replace(" ", "")
    return final_result

In [36]:
model_output = []
for i, result in enumerate(answers):
    try:
        model_output.append(standardize(result))
    except:
        model_output.append("Error")

In [37]:
model_output

['find_card_in_deck,{"rank":"Queen","suit":"Hearts"}',
 'cards.shuffle_and_draw,{"num_cards":3}',
 'poker_game_winner,{"players":["Alex","Sam","Robert","Steve"],"cards":["Aofspades","Kofspades","2ofdiamonds","3ofclubs","Qofhearts","10ofhearts","4ofspades","5ofspades"],"type":"TexasHoldem"}',
 'card_game_probability.calculate,{"total_cards":52,"desired_cards":13,"cards_drawn":1}',
 'poker_probability.full_house,{"deck_size":52,"hand_size":5}']

In [42]:
df = pd.DataFrame()
df["possible_answer"] = possible_answers
df["model_output"] = model_output
df.head()

,possible_answer,model_output
0,"calculate_triangle_area,{""base"":[10],""height"":...","calculate_triangle_area,{""base"":10,""height"":5}"
1,"math.factorial,{""number"":[5]}","math.factorial,{""number"":5}"
2,"math.hypot,{""x"":[4],""y"":[5],""z"":["""",0]}","math.hypot,{""x"":4,""y"":5}"
3,"algebra.quadratic_roots,{""a"":[1],""b"":[-3],""c"":...","algebra.quadratic_roots,{""a"":1,""b"":-3,""c"":2}"
4,"solve_quadratic_equation,{""a"":[2],""b"":[6],""c"":...","solve_quadratic_equation,{""a"":2,""b"":6,""c"":5}"


In [43]:
df.to_csv("results_Outlines_Hermes2Pro.csv", index=False)